In [0]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
# Reshaping to format which CNN expects (batch, height, width, channels)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1).astype('float32')

In [0]:
# normalize inputs from 0-255 to 0-1
X_train/=255
X_test/=255

In [0]:
# one hot encode
number_of_classes = 10
y_train = np_utils.to_categorical(y_train, number_of_classes)
y_test = np_utils.to_categorical(y_test, number_of_classes)


In [6]:
# create model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(number_of_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [8]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 53s 884us/step - loss: 0.3136 - acc: 0.9056 - val_loss: 0.0669 - val_acc: 0.9813
Epoch 2/10
60000/60000 [==============================] - 52s 869us/step - loss: 0.0779 - acc: 0.9757 - val_loss: 0.0482 - val_acc: 0.9843
Epoch 3/10
60000/60000 [==============================] - 52s 872us/step - loss: 0.0551 - acc: 0.9825 - val_loss: 0.0371 - val_acc: 0.9885
Epoch 4/10
60000/60000 [==============================] - 52s 872us/step - loss: 0.0455 - acc: 0.9857 - val_loss: 0.0321 - val_acc: 0.9896
Epoch 5/10
60000/60000 [==============================] - 53s 875us/step - loss: 0.0375 - acc: 0.9882 - val_loss: 0.0285 - val_acc: 0.9904
Epoch 6/10
60000/60000 [==============================] - 52s 869us/step - loss: 0.0332 - acc: 0.9892 - val_loss: 0.0258 - val_acc: 0.9916
Epoch 7/10
60000/60000 [==============================] - 52s 872us/

In [19]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
model.save('digit_recognizer.h5')    
model_file = drive.CreateFile({'title' : 'digit_recognizer.h5'})
model_file.SetContentFile('digit_recognizer.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1k0kO5AT4jLbHYUr68YmLTUclg8c54dl_'})

In [20]:
# Final evaluation of the model
metrics = model.evaluate(X_test, y_test, verbose=0)
print("Metrics(Test loss & Test Accuracy): ")
print(metrics)

Metrics(Test loss & Test Accuracy): 
[0.02811876432190693, 0.991]
